[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

In [2]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [3]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [4]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open('pcr_tests.json', 'rt') as INFILE:
    data=json.load(INFILE)


In [8]:
def parse_date(datestring):
    '''convert a date string into a pandas datetime object'''
    return pd.to_datetime(datestring, format='%Y-%m-%d')

def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    datalist = rawdata['data']
    dates = [dictionary['date'] for dictionary in datalist]
    dates.sort()
    startdate = parse_date(dates[0])
    enddate = parse_date(dates[-1]) 
    index = pd.date_range(startdate,enddate,freq='D')
    timeseriesdf=pd.DataFrame(index=index, columns=['pcr_tests', 'cases']) 
    for entry in datalist: 
        date = parse_date(entry['date']) 
        for column in ['pcr_tests', 'cases']:
            if pd.isna(timeseriesdf.loc[date,column]):
                value =float(entry[column]) if entry[column]!=None else 0.0
                timeseriesdf.loc[date,column]=value

          

    timeseriesdf.fillna(0.0, inplace=True) 
    pcr_vs_cases = timeseriesdf
    return pcr_vs_cases

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df=wrangle_data(data) # df is the dataframe for plotting
print(df)

              pcr_tests       cases
2020-01-31          0.0         2.0
2020-02-01          0.0         2.0
2020-02-02          0.0         2.0
2020-02-03          0.0         2.0
2020-02-04          0.0         2.0
...                 ...         ...
2022-05-16  212281005.0  22203799.0
2022-05-17  212574700.0  22212395.0
2022-05-18  212658028.0  22220178.0
2022-05-19  212754383.0  22232377.0
2022-05-20          0.0  22238713.0

[841 rows x 2 columns]


## Download current data

In [ ]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
from uk_covid19 import Cov19API
import json
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters_pcr = ['areaType=overview']
    structures_pcr = {'date':'date','pcr_tests':'cumPCRTestsByPublishDate', 
                      "cases": "cumCasesByPublishDate"} 
    api = Cov19API(filters=filters_pcr, structure=structures_pcr)
    return api # return data read from the API


In [12]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Refresh', # you may want to change this...
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Keep calm and carry on",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='info', description='Refresh', icon='download', style=ButtonStyle(), tooltip='Keep calm an…

## Graphs and Analysis

Click the options to show the graph

In [13]:
series = wdg.SelectMultiple(
    options=['pcr_tests','cases'],
    values = ['pcr_tests','cases'],
    rows = 2,
    description ='Stats: ',
    disabled = False
)
scale = wdg.RadioButtons(
    options=['Linear', 'Log'], 
    description = 'Scale: ',
    disabled = False
) 

controls=wdg.HBox([series,scale])

def timeseries_graph(gcols,gscale):
    if gscale=='Linear':
        logscale=False
    else:
        logscale=True
    ncols = len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
        plt.show()
    else:
        print('Click to select data for graph')
        print('CTRL + click to select more than one category') 

def plot_pcr_test_cases(df):
    df.plot()
    plt.show() # important! update won't work properly without this
    


def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=controls.value
    if current==controls.options[0]:
        other=controls.options[1]
    else:
        other=controls.options[0]
    controls.value=other # forces the redraw
    controls.value=current # now we can change it back
    
    
graph=wdg.interactive_output(timeseries_graph, {'gcols':series, 'gscale':scale})
    
display(controls, graph)

Output()